In [1]:
# !pip install opencv-python
# !pip install imutils

import cv2 as cv
import os
import numpy as np
import imutils

In [1]:
import cv2 as cv
import os
import numpy as np
import imutils

def cropImg(src, x = 0, y = 0, center = False):
    h = 512
    w = 512
    if center:
        x -= int(w/2)
        y -= int(h/2)
        if x < 0:
            x = 0
        if y < 0:
            y = 0
    return src[y:y+h, x:x+w]

def showResize(src, name = "Resized image", scale_percent = 100, x = 0, y = 0):
    width = int(src.shape[1] * scale_percent / 100)
    height = int(src.shape[0] * scale_percent / 100)
    if x>0 and y >0:
        width = x
        height = y
    dim = (width, height)
    resized = cv.resize(src, dim, interpolation = cv.INTER_AREA)
#     print('Resized Dimensions : ',resized.shape) 
    cv.imshow(name, resized)
    
def conturFilter(src, small = 100, big = 1000):
    cnts = cv.findContours(src.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    mask = np.ones(src.shape[:2], dtype="uint8") * 255
    for c in cnts:
        # if the contour is bad, draw it on the mask
        if cv.contourArea(c) > big or cv.contourArea(c) < small:
            cv.drawContours(mask, [c], -1, 0, -1)
        
    return cv.bitwise_and(src, src, mask=mask)
    
def morphFilter(src, x = 2, y = 2):
    ellipse = cv.getStructuringElement(cv.MORPH_ELLIPSE,(x,y))
    src = cv.morphologyEx(src, cv.MORPH_OPEN, ellipse)
    src = cv.morphologyEx(src, cv.MORPH_CLOSE, ellipse)
    src = conturFilter(src, 100, 500)
    return src

def getBox(src, mask):
    gray = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
#     ret,th1 = cv.threshold(mask,127,255,cv.THRESH_BINARY)
    cnts = cv.findContours(gray.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    for cnt in cnts:
        rect = cv.minAreaRect(cnts[0])
        box = cv.boxPoints(rect)
        box = np.int0(box)
        src = cv.drawContours(src,[box],0,(0,0,255),6)
    return src


def getBears(mask):
    gray = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
    cnts = cv.findContours(gray.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    bears_c = []
    cnts = imutils.grab_contours(cnts)
    
    for cnt in cnts:
        M = cv.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        bears_c += {cx, cy}
    return bears_c    

in_path = "./dataset/0.JPG"

In [ ]:
mask = cv.imread('./masks/0.png')
img = cv.imread(in_path)
bear_c = getBears(mask)

print(bear_c)
img = getBox(img, mask)
crop_img = cropImg(img, bear_c[1], bear_c[0], True)
cv.imshow('bear', crop_img)
showResize(img, 'box', 20, 700, 700)
cv.waitKey(0)
cv.destroyAllWindows()

[136, 2491]


In [2]:

directory = '.\dataset_full'
directory_m = '.\masks'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    filename = filename.replace('.JPG', '.png')
    f_m = os.path.join(directory_m, filename)
    # checking if it is a file
    if not os.path.isfile(f) or not os.path.isfile(f_m):
        print(f)
        print(f_m)
        continue
    img = cv.imread(f)
    img_m = cv.imread(f_m)
    img = cv.medianBlur(img,11)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret,th1 = cv.threshold(gray,127,255,cv.THRESH_BINARY_INV)
    th2 = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV,11,2)
#     th3 = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)


    img = getBox(img, img_m)
#     showResize(img, 'src' + f.split('\\')[-1], 20, 700, 700)
    cv.imwrite("G:/HACK/medved/Crop/out/SRC/"+ f.split('\\')[-1], img)
    
    titles = ['Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding']
#     , 'Adaptive Gaussian Thresholding'
    images = [th1, th2]
#     , th3
    for i in range(len(images)):
        images[i] = morphFilter(images[i], 5, 5)
        rgb_img = cv.cvtColor(images[i], cv.COLOR_GRAY2RGB)
        rgb_img = getBox(rgb_img, img_m)
#         showResize(rgb_img, titles[i], 20, 700, 700)
        cv.imwrite("G:/HACK/medved/Crop/out/BIN/"+ titles[i]+f.split('\\')[-1], rgb_img)
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    

.\dataset_full\12.jpg
.\masks\12.jpg


In [ ]:
# img_m = cv.imread('./masks/0.png')
# gray = cv.cvtColor(img_m,cv.COLOR_BGR2GRAY)
# _,thresh = cv.threshold(gray,1,255,cv.THRESH_BINARY)
# showResize(thresh, 'thresh', 20, 700, 700)

# cnts,_ = cv.findContours(thresh, 1, 2)
# for cnt in cnts:
#     rect = cv.minAreaRect(cnt)
#     box = cv.boxPoints(rect)
#     box = np.int0(box)
#     print(box)
#     cv.drawContours(img_m,[box],0,(0,0,255),3)
    

# showResize(img_m, 'mask', 20, 700, 700)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [ ]:
# image = cv.imread(in_path)
# gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
# edged = cv.Canny(gray, 50, 100)
# ret,th1 = cv.threshold(gray,127,255,cv.THRESH_BINARY)
# th1 = cv.bitwise_not(th1)
# showResize(th1, 'Original', 20)
# # find contours in the image and initialize the mask that will be
# # used to remove the bad contours
# cnts = cv.findContours(th1.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
# cnts = imutils.grab_contours(cnts)
# mask = np.ones(image.shape[:2], dtype="uint8") * 255
# print(len(cnts))
# # loop over the contours
# for c in cnts:
# 	# if the contour is bad, draw it on the mask
# 	if cv.contourArea(c) > 1000 or cv.contourArea(c) < 100:
# 		cv.drawContours(mask, [c], -1, 0, -1)
# # remove the contours from the image and show the resulting images
# image = cv.bitwise_and(th1, th1, mask=mask)
# showResize(mask, 'Mask', 20)
# showResize(image, 'After', 20)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [ ]:

# # out_path = './21_cut.jpg'
# img = cv.imread(in_path)
# showResize(img, 'src', 20)
# img = cv.medianBlur(img,11)
# gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# ret,th1 = cv.threshold(gray,127,255,cv.THRESH_BINARY)
# showResize(th1, 'th1', 20)
# th1 = morphFilter(th1)
# showResize(th1, 'th1_f', 20)
# # crop_img = cropImg(img, 1000)
# # # cv2.imwrite(out_path, crop_img)
# # cv2.imshow("cropped", crop_img)
# cv.waitKey(0)
# cv.destroyAllWindows()